#**Trabalho final - Classificador de produtos com GenAI**

**Nota de atenção:**
- Leia com atenção o descritivo do trabalho e as orientações do template.
- O trabalho deve ser entregue **respeitando a estrutura do arquivo de template**, utilizando o notebook "Template Trabalho final - Classificador de produtos com GenAI.ipynb" e compactado no formato .zip.
- Deve haver apenas um arquivo no formato .ipynb, consolidando todo o trabalho.

**Participantes (RM - NOME):**<br>
João Paulo Martins Rodrigues – RM 364668<br>
Herivelto Raimundo Lemos de Macedo Junior – RM 364212<br>
Evaldo Loiola Mota Junior – RM 364056<br>
Bruno Gomes Nogueira – RM 364457<br>

###**Caso de uso:**

Uma empresa de marketplace, disponibiliza sua plataforma para diversos vendedores cadastrarem seus produtos em diferentes categorias previamente definidas. Essas categorias são utilizadas para melhor distribuir e divulgar seus produtos para os clientes e usuários da plataforma. Mas nem todos os vendedores respeitam essas categorias, regras e as diretrizes do marketplace.

Pense nos diversos problemas que podemos enfrentar:
- Vendedores que cadastram produtos em categorias erradas;
- Vendedores que querem vender produtos ilícitos;
- Vendedores que querem vender produtos que não são permitidos pelas políticas do marketplace e por aí vai...

**Será que é possível validar o cadastro desses produtos e categorias, produto por produto**? Um por um?... Que trampo, não???

###**Desafio:**
- Conseguimos ajudar a mitigar parte desse problema?
- Você conseguiria criar algum **processo** que diminua esse trabalho manual e que **valide a categoria** desses produtos?

Bom, podemos criar um **processo que seja capaz de classificar um produto através do nome e da descrição**, e depois podemos confrontar com a categoria e premissas da plataforma usando **IA Generativa e modelos LLMs**.

###**Orientações:**

---
####**Usem o Google Colab com Python e esse template para desenvolverem o trabalho.**
---

**1. Análise exploratória**, vamos começar explorando a base de dados de produtos [1]?
  - Faça uma análise exploratória para entender os dados e estrutura do dataframe.
  - Crie uma nova coluna no dataframe chamada **`texto`** concatenando as colunas **`nome`** e **`descricao`**. Ex.: `nome + " " + descricao`.
  - Crie um dataframe novo e selecionando uma amostra aleatória de **100 registros** usando o `random_state = 42` (veja o exemplo abaixo).
  
  ***Obs**.:
    - É importante manter essa configuração da amostra para os trabalhos ficarem comparáveis e classificando os mesmos produtos.
    - Faça a análise exploratória com a base completa e não apenas com o sample.

**2. Desenvolvimento e testes**, nessa parte é onde vocês podem explorar o desenvolvimento do trabalho aplicando as técnicas, ferramentas e serviços de GenAI.  
  - Explore diferentes formas de tratar o problema. Comece selecionando casos individuais para testar as ferramentas, framework, APIs e técnicas de prompt engineering.
  - Fiquem à vontade para explorar os serviços e frameworks vistos em aula: API da OpenAI Platform, API da Azure AI Foundry e LangChain.
  - Sejam criativo, mas não precisa de complexidade e podem explorar outras formas de desenvolver.
  - Explique as decisões e racional do desenvolvimento. Abuse dos comentários.

**3. Processo final**, aqui nessa parte separe apenas o processo final com um pipeline completo para classificar a categoria dos produtos.
  - Crie uma função de receba um `texto` (nome + descrição do produto) e retorne a categoria classificada pelo texto.
  - Aplique essa função no dataframe com os 100 casos.
  - Resultado esperado: No dataframe de produto com os 100 casos, crie uma nova coluna de categoria: `categoria_genai`.
  - Valide se a classificação da Ia Generativa de cada produto está correta ou divergente comparando com a coluna `categora` escolhida pelo vendedor. Crie uma nova coluna `validacao_categoria` com os valores da validação: `correta` ou `divergente`.

###**Avaliação:**
O trabalho será avaliado pelas seguintes diretrizes:
  - Demonstração de conhecimento com os temas abordados em sala de aula.
  - Utilização correta dos frameworks, APIs e aplicação das técnicas de prompt engineering.
  - Organização, comentários e explicação certamente vão ajudar na nota.
  - Resultado esperado seguindo as orientações do professor nesse template.

###**Atenção:**
- Use sua conta da **Azure AI Foundry** ou da **OpenAI Platform** para desenvolver o trabalho, mas dê preferência para a conta da Azure por causa dos limites de crédito. **Não deixe suas credenciais no trabalho, por favor!**
- Trabalhos iguais são passíveis de reprovação ou desconto de nota.
- Respeite a estrutura do template fornecido pelo professor.
- Limite de 4 pessoas por grupo, de preferência o mesmo grupo do Startup One.

###**[1] Base de dados de produtos - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv**


##**1. Análise exploratória**

Desenvolva aqui:

In [1]:
import pandas as pd
import re
import numpy as np

df_raw = pd.read_csv(
    "https://dados-ml-pln.s3-sa-east-1.amazonaws.com/produtos.csv",
    delimiter=";",
    encoding='utf-8' ).sample(100, random_state=42)

df_raw.head()

,nome,descricao,categoria
33,Extraordinário,Produto Novo“Extraordinário” é um livro que co...,livro
3316,Fifa 2018 Narração Português Completo Midia D...,FIFA 2018 - XBOX 360 MIDIA DIGITAL COMPLETOATE...,game
1557,Kit Mega Sarutobi Kunai Naruto Asuma Shuriken...,Descrição do Kit Ninja (Foto):Uma Kunai Asuma ...,brinquedo
2548,Caixa 50 Cores Batom Queen Fosco Matte Nude K...,CAIXA C/ 50 CORES DIFERENTES!BATOM MATTE QUEEN...,maquiagem
457,Box Dourado Crónica De Gelo E Fogo Edição De ...,A série Crônicas de Gelo e apresentações. São ...,livro


In [2]:
#analisando as categorias do dataframe original
df_cat = df_raw[['nome','categoria']].groupby('categoria').count()
df_cat

,nome
categoria,
brinquedo,28
game,30
livro,21
maquiagem,21


##**2. Desenvolvimento e testes**

Desenvolva aqui:

In [3]:
#criação de id para futuro merge de comparação
df_raw = df_raw.reset_index().rename(columns={"index": "id"}).astype(str)
df_raw.head()

,id,nome,descricao,categoria
0,33,Extraordinário,Produto Novo“Extraordinário” é um livro que co...,livro
1,3316,Fifa 2018 Narração Português Completo Midia D...,FIFA 2018 - XBOX 360 MIDIA DIGITAL COMPLETOATE...,game
2,1557,Kit Mega Sarutobi Kunai Naruto Asuma Shuriken...,Descrição do Kit Ninja (Foto):Uma Kunai Asuma ...,brinquedo
3,2548,Caixa 50 Cores Batom Queen Fosco Matte Nude K...,CAIXA C/ 50 CORES DIFERENTES!BATOM MATTE QUEEN...,maquiagem
4,457,Box Dourado Crónica De Gelo E Fogo Edição De ...,A série Crônicas de Gelo e apresentações. São ...,livro


In [4]:
df_teste = df_raw[df_raw['id'] == '1377']
df_teste.head()

,id,nome,descricao,categoria
12,1377,Kit 2 Pelucia Super Mario Bros E Luigi Games ...,nan,brinquedo


In [5]:
df_teste.dtypes

id           object
nome         object
descricao    object
categoria    object
dtype: object

In [6]:
#vendo que possuem colunas com descrição nan, colocando como desconhecido
df_analise = df_raw.copy()
df_analise["descricao"] = np.where(df_analise["descricao"] == 'nan', 'Desconhecido', df_analise["descricao"])

In [7]:
#criando df para analise com coluna id e texto para input no chatgpt
df_analise['texto'] = df_analise['nome']  + " - " + df_analise['descricao']
df_analise.drop(columns=['nome','descricao','categoria'], inplace=True)
df_analise.count()

id       100
texto    100
dtype: int64

In [8]:
#selecionando apenas 1 linha de input para teste, id selecionado foi especifico para o desenvolvimento do script, pois estava dando divergencias na classificação
#assim foi feito o tunning para total compatibilidade

df_teste = df_analise[df_analise['id'] == '1377']
df_teste.head()

,id,texto
12,1377,Kit 2 Pelucia Super Mario Bros E Luigi Games ...


In [ ]:
#importação de bibliotecas utilizando langchain e criação da llm

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel
from pydantic import TypeAdapter
from langchain_core.runnables import RunnableLambda
import numpy as np
import time
import pandas as pd
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import re

import os
from openai import AzureOpenAI

azure_endpoint = ""
deployment_name = ""  
api_key = ""           
api_version = ""     

llm = AzureChatOpenAI(
    azure_deployment=deployment_name,
    api_version=api_version,
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    temperature=0
)

prompt = ChatPromptTemplate.from_template("Responda em português: {pergunta}")
chain = LLMChain(llm=llm, prompt=prompt)
resposta = chain.invoke({"pergunta": "Qual é voce?"})
print(resposta["text"])

C:\Users\JOAO PC\AppData\Local\Temp\ipykernel_32080\2358783889.py:35: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


Eu sou um assistente virtual, projetado para ajudar a responder perguntas e fornecer informações sobre uma variedade de tópicos. Como posso ajudar você hoje?


In [10]:
#criacao do system e promt e submit da requisicao

SYSTEM = """
Você é um classificador de produtos. Sua única tarefa é ler o **texto** de cada item do JSON e atribuir **uma única** categoria dentre:
- brinquedo
- game
- livro
- maquiagem
- outros

REGRAS GERAIS:
- Você recebe um JSON com os campos "id" e "texto".
- Use **apenas** as informações fornecidas no campo "texto". Não invente dados.
- O campo "texto" é a concatenação de "nome - descricao".
- Se a descrição estiver ausente, nula ou igual a "nan" ou "Desconhecido", **considere apenas o nome para classificar**.
- Responda **apenas** com JSON válido.
- Para cada item, retorne exatamente os campos: "id", "categoria_genai".
- "id" é o identificador do item e deve ser mantido sem alterações.
- "categoria_genai" deve ser **exatamente** um destes valores: "brinquedo", "game", "livro", "maquiagem", "outros".
- Não inclua campos extras, comentários, explicações ou raciocínio.

HEURÍSTICAS DE CATEGORIZAÇÃO:
1) **game**
   - Palavras/termos: jogo, videogame, video game, game, PS5, PlayStation, Xbox, Nintendo, Switch, Steam, PC game, console, joystick, controle, DLC.
   - Frases como: "mídia digital", "código de resgate", "cartucho", "disco de jogo", faça inferencia de personagens, mas considerando que a descrição trate de algo eletronico ou de cartas, joggos de tabuleiro, baralhos, rpg de mesa.

2) **livro**
   - Palavras/termos: livro, romance, HQ, graphic novel, mangá, guia, manual, apostila, edição, capa dura, brochura, ISBN, autor, editora, coleção.

3) **maquiagem**
   - Palavras/termos: batom, gloss, rímel, máscara de cílios, delineador, sombra, base, corretivo, pó, blush, iluminador, primer, paleta, pincel de maquiagem.
   - Referências a marcas cosméticas, cores/tons, acabamentos (matte, glow), FPS quando ligado a produtos de make.

4) **brinquedo**
   - Palavras/termos: brinquedo, boneca, boneco, carrinho, playset, quebra-cabeça, puzzle, jogo de tabuleiro infantil, pelúcia, LEGO, blocos de montar, slime, massinha, pelucia, estatura, action figure.
   - Itens infantis de caráter lúdico, sem evidência clara de serem jogos digitais, tente inferir se há alguma evidencia de personagens não sendo jogos eletronicos.

5) **outros**
   - Use quando o item não se encaixar em **nenhuma** das categorias anteriores.
   - Exemplos: roupas, eletrônicos (celulares, TV, fone de ouvido, notebook), eletrodomésticos, utensílios domésticos, acessórios, alimentos, bebidas, perfumes, produtos de skincare (creme, hidratante, sabonete, protetor solar quando NÃO for maquiagem), produtos de higiene, móveis, papelaria, ferramentas.
   - Também use se o texto for **muito vago ou genérico**, sem dar evidência suficiente para outra categoria (ex.: "Produto variado", "Item promocional", "Kit sortido").
   - Se o texto indicar ausência de informação (como "nan", "não informado", "sem descrição") → use **apenas o nome** para classificar.

DESEMPATE:
- Se houver forte evidência de uma categoria, escolha-a.
- Priorize as palavras-chave listadas nas heurísticas.
- Se o item mencionar videogame/jogo digital → "game".
- Se mencionar ISBN, autor ou editora → "livro".
- Se mencionar claramente cosméticos de maquiagem → "maquiagem".
- Se mencionar vocabulário infantil lúdico → "brinquedo".
- Se o texto indicar ausência de descrição → classifique com base apenas no nome.
- Caso não se enquadre em nenhuma das regras acima → "outros".

FORMATOS DE ENTRADA ESPERADOS:
- Lista JSON em que cada item possui: "id" (string), "texto" (string).

FORMATOS DE SAÍDA:
- Saída: sempre uma lista JSON de objetos.
- Cada objeto de saída deve ter exatamente:
  - "id" (mesmo valor recebido na entrada, sem alterações)
  - "categoria_genai" (uma das quatro categorias listadas acima)
- JSON (lista). Cada item deve ser:
{{
  "id": string,
  "categoria_genai": string
}}
"""

PROMPT_ALL = ChatPromptTemplate.from_messages([
    ("system", SYSTEM),
    ("user", "Classifique os itens a seguir (lista JSON com objetos):\n\n{items}\n\nResponda apenas com JSON válido (lista).")
])



class ItemOut(BaseModel):
    id: str
    categoria_genai: str


def validar_saida(resp_content):
    adapter = TypeAdapter(list[ItemOut])
    models = adapter.validate_json(resp_content)   
    return pd.DataFrame(adapter.dump_python(models))

def make_item_injector(row):
    record = {"id": str(row["id"]), "texto": str(row["texto"])}
    items_json = json.dumps([record], ensure_ascii=False) 
    return RunnableLambda(lambda _: {"items": items_json})

def build_chain_for_row(row):
    return (
        make_item_injector(row)
        | PROMPT_ALL
        | llm
        | (lambda msg: msg.content)
        | RunnableLambda(lambda text: validar_saida(text))
    )

def processar_linhas(df):
    results = []
    i = 0
    for _, row in df.iterrows():
        chain = build_chain_for_row(row)
        r = chain.invoke({})
        i += 1
        print(r)
        print(i)
        results.append(r)
    final_df = pd.concat(results, ignore_index=True)
    return final_df



In [11]:
teste = processar_linhas(df_analise.sample(5))

     id categoria_genai
0  3484            game
1
     id categoria_genai
0  1517       brinquedo
2
    id categoria_genai
0  109           livro
3
    id categoria_genai
0  296           livro
4
     id categoria_genai
0  3880            game
5


In [12]:
df_comp = df_raw.copy()
df_comp = df_comp[['id', 'nome','descricao', 'categoria']]

comparar = pd.merge(teste, df_comp, how='left', on='id')
comparar['validacao_categoria'] = np.where(comparar['categoria_genai'] == comparar['categoria'], "correta", "divergente")

cols_orders = ['id', 'categoria', 'categoria_genai', 'validacao_categoria', 'nome','descricao']
comparar = comparar[cols_orders]

divergentes = comparar[comparar['validacao_categoria'] == "divergente"]

In [13]:
divergentes

,id,categoria,categoria_genai,validacao_categoria,nome,descricao


##**3. Processo final**

Desenvolva aqui:

In [14]:
resultado = processar_linhas(df_analise)
resultado.count()

   id categoria_genai
0  33           livro
1
     id categoria_genai
0  3316            game
2
     id categoria_genai
0  1557       brinquedo
3
     id categoria_genai
0  2548       maquiagem
4
    id categoria_genai
0  457           livro
5
     id categoria_genai
0  1444       brinquedo
6
     id categoria_genai
0  1029       brinquedo
7
     id categoria_genai
0  3880            game
8
     id categoria_genai
0  4005            game
9
     id categoria_genai
0  1736       brinquedo
10
     id categoria_genai
0  3852            game
11
    id categoria_genai
0  495           livro
12
     id categoria_genai
0  1377       brinquedo
13
     id categoria_genai
0  1258       brinquedo
14
     id categoria_genai
0  2760       maquiagem
15
    id categoria_genai
0  564           livro
16
     id categoria_genai
0  1952       brinquedo
17
    id categoria_genai
0  568           livro
18
     id categoria_genai
0  1885       brinquedo
19
    id categoria_genai
0  881           livro
20
   

id                 100
categoria_genai    100
dtype: int64

In [15]:
df_comp = df_raw.copy()
df_comp = df_comp[['id', 'nome','descricao', 'categoria']]

comparar = pd.merge(resultado, df_comp, how='left', on='id')
comparar['validacao_categoria'] = np.where(comparar['categoria_genai'] == comparar['categoria'], "correta", "divergente")

cols_orders = ['id', 'categoria', 'categoria_genai', 'validacao_categoria', 'nome','descricao']
comparar = comparar[cols_orders]

divergentes = comparar[comparar['validacao_categoria'] == "divergente"]

In [16]:
comparar.head()

,id,categoria,categoria_genai,validacao_categoria,nome,descricao
0,33,livro,livro,correta,Extraordinário,Produto Novo“Extraordinário” é um livro que co...
1,3316,game,game,correta,Fifa 2018 Narração Português Completo Midia D...,FIFA 2018 - XBOX 360 MIDIA DIGITAL COMPLETOATE...
2,1557,brinquedo,brinquedo,correta,Kit Mega Sarutobi Kunai Naruto Asuma Shuriken...,Descrição do Kit Ninja (Foto):Uma Kunai Asuma ...
3,2548,maquiagem,maquiagem,correta,Caixa 50 Cores Batom Queen Fosco Matte Nude K...,CAIXA C/ 50 CORES DIFERENTES!BATOM MATTE QUEEN...
4,457,livro,livro,correta,Box Dourado Crónica De Gelo E Fogo Edição De ...,A série Crônicas de Gelo e apresentações. São ...


In [17]:
divergentes.count()

id                     3
categoria              3
categoria_genai        3
validacao_categoria    3
nome                   3
descricao              3
dtype: int64

In [18]:

divergentes

,id,categoria,categoria_genai,validacao_categoria,nome,descricao
21,3318,game,outros,divergente,"2.000 Gold Wow Gold Ouro Azralon (nemesis 2,0...",Apos a compra o mercadolivre fornecera dados c...
22,1499,brinquedo,outros,divergente,Expositor Estante Figures Eaglemoss Marvel St...,nan
83,3106,game,outros,divergente,Acabamento Para Degrau De Escada - Cantoneira...,Cantoneira L P/ Acabamento de Degrau cor.: Alu...
